<a href="https://colab.research.google.com/github/rupaut98/LLM_Oreilly/blob/main/ch1_ch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
"microsoft/Phi-3-mini-4k-instruct",
device_map="cuda",
torch_dtype="auto",
trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

In [ ]:
prompt = "<s> Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"
# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
# Generate the text
generation_output = model.generate(
input_ids=input_ids,
max_new_tokens=20
)
# Print the output
print(tokenizer.decode(generation_output[0]))

In [ ]:
for id in input_ids[0]:
  print(tokenizer.decode(id))

In [ ]:
print(generation_output)

In [ ]:
print(tokenizer.decode([278,25305, 293, 16423]))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

In [ ]:
text = """
English and CAPITALIZATION
🎵鸟
show_tokens False None elif == >= else: two tabs:" " Three tabs: " "
12.0*50=600
"""

In [ ]:
colors_list = [
'102;194;165', '252;141;98', '141;160;203',
'231;138;195', '166;216;84', '255;217;47'
]
def show_tokens(sentence, tokenizer_name):
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
  token_ids = tokenizer(sentence).input_ids
  for idx, t in enumerate(token_ids):
    print(
    f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
    tokenizer.decode(t) +
    '\x1b[0m',
    end=' '
    )

In [ ]:

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer):
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode([t]) +
            '\x1b[0m',
            end=' '
        )

# Input text
text = """
English and CAPITALIZATION
🎵鸟
show_tokens False None elif == >= else: two tabs:" " Three tabs: " "
12.0*50=600
"""

# Show tokens for the input text
show_tokens(text, tokenizer)


In [ ]:
from transformers import AutoModel, AutoTokenizer
# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
# Load a language model
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")
# Tokenize the sentence
tokens = tokenizer('Hello world', return_tensors='pt')
# Process the tokens
output = model(**tokens)[0]

In [ ]:
print(output)

In [ ]:
for token in tokens['input_ids'][0]:
  print(tokenizer.decode(token))

In [ ]:
!pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer
# Load model
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
# Convert text to text embeddings
vector = model.encode("Best movie ever!")

In [ ]:
vector.shape

In [ ]:
import pandas as pd
from urllib import request
# Get the playlist dataset file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')
# Parse the playlist dataset file. Skip the first two lines as
# they only contain metadata
lines = data.read().decode("utf-8").split('\n')[2:]
# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]
# Load song metadata
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [ ]:
print( 'Playlist #1:\n ', playlists[0], '\n')
print( 'Playlist #2:\n ', playlists[1])

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
"microsoft/Phi-3-mini-4k-instruct"
,
device_map=
"cuda"
,
torch_dtype=
"auto"
,
trust_remote_code=True,
)

generator = pipeline(
    'text-generation',
    model = model,
    tokenizer = tokenizer,
    return_full_text = False,
    max_new_tokens = 50,
)

In [ ]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happen"
# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

In [ ]:
%%timeit -n 1
# Generate the text
generation_output = model.generate(
input_ids=input_ids,
max_new_tokens=50,
use_cache=False
)

In [ ]:
!pip install datasets
from datasets import load_dataset

In [ ]:
data = load_dataset("rotten_tomatoes")
data

In [ ]:
data["train"][0, -1]

In [ ]:
# Using a pretrained model to classify sentiments

from transformers import pipeline

model_path = 'cardiffnlp/twitter-roberta-base-sentiment-latest'

pipe = pipeline(
    model = model_path,
    tokenizer = model_path,
    return_all_scores = True,
    device = "cuda:0"
)

In [ ]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

y_pred = []

for output in tqdm(pipe(KeyDataset(data["test"], "text")), total = len(data["test"])):
  negative_score = output[0]["score"]
  positive_score = output[2]["score"]
  assignment = np.argmax([negative_score, positive_score])
  y_pred.append(assignment)

In [ ]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, _pred):

  performance = classification_report(
      y_true, y_pred,
      target_names = ["Negative Review", "Positive Review"]
  )
  print(performance)


In [ ]:
#What if we don't have a pretrained model for sentiments. Well, we don't really need to finetune the model and train it which would be
#computationally expensive, rather we can use a pretrained embedding model like BERT or sentence-transformers
#and then use it to train a classifier model using logistic regression thus using CPU instead of GPU
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

train_embeddings = model.encode(data["train"]["text"], show_progress_in_bar = True)
test_embeddings = model.encode(data["test"]["text"], show_progress_in_bar = True)

In [ ]:
train_embeddings[0].shape

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state = 42)
clf.fit(train_embeddings, data["train"]["label"])


In [ ]:
y_pred = clf.predict(test_embeddings)
evaluate_performance(data["test"]["label"], y_pred)

In [ ]:
label_embeddings = model.encode(["A negative review", "A positive review"])

In [ ]:
label_embeddings[0].shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
sim_matrix[0]
y_pred = np.argmax(sim_matrix, axis = 1)

In [ ]:
sim_matrix[0]

In [ ]:
y_pred[0]

In [ ]:
evaluate_performance(data["test"]["label"], y_pred)

In [ ]:
#Using a generative model (with both decoder and encoding layers) instead of just models with encoding lyaers only
from transformers import pipeline
pipe = pipeline(
    "text2text-generation",
    model = "google/flan-t5-small",
    device = "cuda:0"
)

In [ ]:
prompt = "Is the following sentence positive or negative"
data = data.map(lambda example: {"t5": prompt + example['text']})
data["train"][0]

In [ ]:
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset
y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"], "t5")), total = len(data["test"])):
  print(output)
  text = output[0]["generated_text"]
  y_pred.append(0 if text == "negative" else 1)

In [ ]:
evaluate_performance(data["test"]["label"], y_pred)

In [ ]:
!pip install openai

In [ ]:
import openai
client = openai.OpenAI(api_key=api_key)

In [ ]:
def chatgpt_generation(prompt, document, model = "gpt-3.5-turbo-0125"):
  messages = [
      {
          "role": "system",
          "content": "You are a helpful assistant"
        },
      {
          "role": "user",
          "content": prompt.replace("[DOCUMENT]", document)
      }
  ]

  chat_completion = client.chat.completions.create(
      messages = messages,
      model = model,
      temperature = 0
  )
  return chat_completion.choices[0].message.content

In [ ]:
prompt = """Predict whether the following document is a positive or negative movie review:
[DOCUMENT]
If it is positive return 1 and if it is negative return 0. Do not give any other answers.
"""
document = "unpretentious , charming , quirky , original"
chatgpt_generation(prompt, document)